In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np

from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=200)

It took 00 mins and 00 seconds to download 200 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    long_or_short=LongOrShortType.Long,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_equity=1000.0,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=ZeroOrEntryType.Nothing,
)


In [5]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 2
Total combinations of settings to test: 2
Total candles: 200
Total candles to test: 400


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,7.0,1,6,-19.020,14.286,-0.0,150.311,-190.199,809.801
1,0,1,7.0,1,6,-18.375,14.286,-0.0,170.206,-183.747,816.253


In [6]:
strat_records_df.sort_values(by='to_the_upside', ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,7.0,1,6,-19.020,14.286,-0.0,150.311,-190.199,809.801
1,0,1,7.0,1,6,-18.375,14.286,-0.0,170.206,-183.747,816.253


In [7]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=1,
)

In [8]:
data

{'candles': [[1699179300000.0, 35063.8, 35080.8, 35050.1, 35077.4],
  [1699179600000.0, 35077.4, 35083.3, 35070.8, 35072.9],
  [1699179900000.0, 35072.9, 35078.6, 35061.1, 35075.0],
  [1699180200000.0, 35075.0, 35110.2, 35067.7, 35107.4],
  [1699180500000.0, 35107.4, 35125.7, 35104.5, 35108.2],
  [1699180800000.0, 35108.2, 35119.5, 35097.0, 35119.4],
  [1699181100000.0, 35119.4, 35162.3, 35117.0, 35156.7],
  [1699181400000.0, 35156.7, 35162.3, 35140.8, 35151.7],
  [1699181700000.0, 35151.7, 35155.2, 35140.1, 35143.3],
  [1699182000000.0, 35143.3, 35156.3, 35140.3, 35155.2],
  [1699182300000.0, 35155.2, 35155.9, 35125.0, 35130.0],
  [1699182600000.0, 35130.0, 35142.8, 35123.6, 35131.0],
  [1699182900000.0, 35131.0, 35134.3, 35117.4, 35122.5],
  [1699183200000.0, 35122.5, 35155.8, 35122.5, 35149.1],
  [1699183500000.0, 35149.1, 35177.0, 35142.1, 35145.4],
  [1699183800000.0, 35145.4, 35161.4, 35141.0, 35141.0],
  [1699184100000.0, 35141.0, 35149.5, 35118.5, 35137.5],
  [1699184400000.0, 

In [ ]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

returned_data = res.json()

print(returned_data)